In [4]:
	paramSetting={'SPY': [{'high':87, 'low':40, 'windowSize':20},{'high':87, 'low':40, 'windowSize':20}],
					'IAU': {'high':83, 'low':41, 'windowSize':25},
					'LQD': {'high':98, 'low':44, 'windowSize':9},
					'DSI': {'high':85, 'low':42, 'windowSize':20}}

In [9]:
print((paramSetting['SPY'][0]['high']))

87


In [15]:
print('\'high\':')

'high':


In [17]:
import sys
import numpy as np
import pandas as pd
def RSI(pastPriceVec, windowSize):
    up_down = np.zeros(windowSize)
    for i in range(1,windowSize):
        up_down[i] = pastPriceVec[-windowSize + i] - pastPriceVec[-windowSize + i - 1]
    rsi = up = down = 0
    for i in up_down:
        if i > 0:
            up += i
        else:
            down -= i
    if up + down != 0:
        rsi = up/(up + down) * 100
    return rsi
# Search for 3 modifiable parameters in RSI(windowSize, high, low,)
def myStrategy(pastPriceVec, currentPrice, windowSize, high, low):
    action = 0
    dataLen = len(pastPriceVec)
    if dataLen == 0:
        return 0
    
    if dataLen < windowSize:
        windowSize = dataLen
    rsi = RSI(pastPriceVec, windowSize)
    if 50 < rsi < high or rsi < low:
        action = 1
    elif low < rsi < 50 or rsi > high:
        action = -1
    else:
        action = 0
    return action
# Compute return rate over a given price vector, with 3 modifiable parameters
def computeReturnRate(priceVec, windowSize, high, low):
	capital=1000	# Initial available capital
	capitalOrig=capital	 # original capital
	dataCount=len(priceVec)				# day size
	suggestedAction=np.zeros((dataCount,1))	# Vec of suggested actions
	stockHolding=np.zeros((dataCount,1))  	# Vec of stock holdings
	total=np.zeros((dataCount,1))	 	# Vec of total asset
	realAction=np.zeros((dataCount,1))	# Real action, which might be different from suggested action. For instance, when the suggested action is 1 (buy) but you don't have any capital, then the real action is 0 (hold, or do nothing). 
	# Run through each day
	for ic in range(dataCount):
		currentPrice=priceVec[ic]	# current price
		suggestedAction[ic]=myStrategy(priceVec[0:ic], currentPrice, windowSize, high, low)		# Obtain the suggested action
		# get real action by suggested action
		if ic>0:
			stockHolding[ic]=stockHolding[ic-1]	# The stock holding from the previous day
		if suggestedAction[ic]==1:	# Suggested action is "buy"
			if stockHolding[ic]==0:		# "buy" only if you don't have stock holding
				stockHolding[ic]=capital/currentPrice # Buy stock using cash
				capital=0	# Cash
				realAction[ic]=1
		elif suggestedAction[ic]==-1:	# Suggested action is "sell"
			if stockHolding[ic]>0:		# "sell" only if you have stock holding
				capital=stockHolding[ic]*currentPrice # Sell stock to have cash
				stockHolding[ic]=0	# Stocking holding
				realAction[ic]=-1
		elif suggestedAction[ic]==0:	# No action
			realAction[ic]=0
		else:
			assert False
		total[ic]=capital+stockHolding[ic]*currentPrice	# Total asset, including stock holding and cash 
	returnRate=(total[-1]-capitalOrig)/capitalOrig		# Return rate of this run
	return returnRate

returnRateBest=-1.00	 # Initial best return rate
df = pd.read_csv('IAU.csv')	# read stock file
adjCloseAll = df["Adj Close"].values		# get adj close as the price vector
windowSizeMin = 10; windowSizeMax = 20;	# Range of windowSize to explore
alphaMin = 60; alphaMax = 80;			# Range of alpha to explore
betaMin = 10; betaMax = 45				# Range of beta to explore
# Start exhaustive search
N = len(adjCloseAll)//240
print(N)
for j in range(N+1):
	returnRateBest=-1.00
	if len(adjCloseAll) >= len(adjCloseAll)//10*(j+1):
		adjClose = adjCloseAll[240 * j: 240 *(j+1)]
	else:
		adjClose = adjCloseAll[240 * j: ]
	for windowSize in range(windowSizeMin, windowSizeMax+1):		# For-loop for windowSize
		#print("windowSize=%d" %(windowSize))
		for alpha in range(alphaMin, alphaMax+1):	    	# For-loop for alpha
			#print("\talpha=%d" %(alpha))
			for beta in range(betaMin, betaMax+1):		# For-loop for beta
				#print("\t\tbeta=%d" %(beta), end="")	# No newline
				returnRate=computeReturnRate(adjClose, windowSize, alpha, beta)		# Start the whole run with the given parameters
				#print(" ==> returnRate=%f " %(returnRate))
				if returnRate > returnRateBest:		# Keep the best parameters
					windowSizeBest=windowSize
					alphaBest=alpha
					betaBest=beta
					returnRateBest=returnRate
	print('{','\'high\':', alphaBest, ',', '\'low\':', betaBest,',', '\'windowSize\':', windowSizeBest,'},')    


15


KeyboardInterrupt: 